## 1. Data wrangling

## 1.1 Contents<a id='2.1_Contents'></a>
* [1 Data wrangling](#2_Data_wrangling)
  * [1.1 Contents](#2.1_Contents)
  * [1.2 Introduction](#2.2_Introduction)
  * [1.3 Imports](#2.3_Imports)
  * [1.4 Load The House Price Data](#2.4_Load_The_House_Price_Data)
  * [1.5 Filtering Single Family Property Type](#2.5_Filtering_Single_Family_Property_Type) 
  * [1.6 Missing Values](#2.6_Missing_Values) 
    * [1.6.1 Features With > 90% Missing Values](#1.6.1_Features_With_>90%_Missing_Values)
    * [1.6.2 Features With 80%-90% Missing Values](#1.6.2_Features_With_80%_-_90%_Missing_Values)
    * [1.6.3 Features With 70%-80% Missing Values](#1.6.3_Features_With_70%_-_80%_Missing_Values)
    * [1.6.4 Features With 50%-70% Missing Values](#1.6.4_Features_With_50%_-_70%_Missing_Values)
    * [1.6.5 Features With 10%-50% Missing Values](#1.6.5_Features_With_10%_-_50%_Missing_Values)
      * [1.6.5.1 Garage](#1.6.5.1_Garage)
      * [1.6.5.2 Living](#1.6.5.2_Living) 
      * [1.6.5.3 Dining](#1.6.5.3_Dining) 
      * [1.6.5.4 Kitchen](#1.6.5.4_Kitchen)
  * [1.7 Subdivisions And Their Facts](#1.7_Subdivisions_And_their_Facts)
  * [1.8 Fill Null For Kitchen, Dining and Living](#1.8_Fill_Null_For_Kitchen_Dining_Living)
  * [1.9 Listing Price](#1.9_Listing_Price)
  * [1.10 Bedrooms](#1.10_Bedrooms)     
  * [1.11 Bathrooms](#1.11_Bathrooms)
  * [1.12 Stories](#1.12_Stories)
  * [1.13 Style](#1.13_Style)
  * [1.14 Year Built](#1.14_Year_Built)
  * [1.15 Building Sqft](#1.15_Building_Sqft)
  * [1.16 Lot Size](#1.16_Lot_Size)
  * [1.17 Maintenance Fee](#1.17_Maintenance_Fee)
  * [1.18 Fireplace](#1.18_Fireplace)
  * [1.19 HOA Mandatory](#1.19_HOA_Mandatory)
  * [1.20 Other Fees](#1.20_Other_Fees)
  * [1.21 Roof](#1.21_Roof)

## 2.2 Introduction<a id='2.2_Introduction'></a>

In this section I will investigate data scrapped from www.HAR.com. Data cleaning will be done in this stage since all rows are categorical and need to be numerical. I will remove features with lost of none values and will create new features.

## 2.3 Imports<a id='2.3_Imports'></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import substring
import os
import re
from tqdm import tqdm
from fuzzywuzzy import process
import warnings
warnings.filterwarnings('ignore')

## 2.4 Load The House Price Data<a id='2.4_Load_The_House_Price_Data'></a>

In [2]:
data= pd.read_csv('../data/raw/Houston_Home_List.csv',encoding = "ISO-8859-1")
print('data shape is:',data.shape)

data shape is: (15065, 101)


In [3]:
data.columns

Index(['Unnamed: 0', 'image_link', 'Listing Price:', 'Address:', 'City:',
       'State:', 'Zip Code:', 'County:', 'Subdivision:', 'Legal Description:',
       ...
       'Extra Room:', 'Wine Room:', 'Carport Description:',
       'Median Appraised Value / Square ft.:', 'Den:', 'Utility Room Desc:',
       'Sunroom:', 'Guest Suite:', 'Bath:', 'Garage Apartment:'],
      dtype='object', length=101)

## 2.5 Filtering Single Family Property Type<a id='2.5_Filtering_Single_Family_Property_Type'></a>

Since we are going to analysis images and other house features it is important to have all records as same as possible. For example for lots there is no image for building or rooms and features for multi-family properties are different from single family homes. let see what kind of property type we have in our dataset:

In [4]:
data['Property Type:'].value_counts()

Single-Family                          11034
Lots                                    1591
Townhouse/Condo - Townhouse              966
Townhouse/Condo - Condominium            600
Mid/Hi-Rise Condo                        444
Country Homes/Acreage                    146
Multi-Family - Duplex                    117
Country Homes/Acreage - Free Standi       47
Multi-Family - Fourplex                   40
Multi-Family - 5 Plus                     37
Multi-Family - Triplex                    18
Multi-Family - Multiple Detached Dw       12
Lot & Acreage - Residential                5
Residential - Condo                        2
Country Homes/Acreage - Manufacture        2
Single Family                              1
Residential - Townhouse                    1
Name: Property Type:, dtype: int64

Majority of properties are single family so, I keep them and remove the rest of the types.

In [5]:
single_family_df = data[data['Property Type:']=='Single-Family']
single_family_df.reset_index(drop=True,inplace=True)
len(single_family_df)

11034

In [6]:
single_family_df.head()

,Unnamed: 0,image_link,Listing Price:,Address:,City:,State:,Zip Code:,County:,Subdivision:,Legal Description:,...,Extra Room:,Wine Room:,Carport Description:,Median Appraised Value / Square ft.:,Den:,Utility Room Desc:,Sunroom:,Guest Suite:,Bath:,Garage Apartment:
0,0,['https://photos.harstatic.com/190420550/hr/im...,"$ 465,000 ($221.85/sqft.) $Convert",110 Pierce Street,Houston,TX,77002,Harris County,Modern Midtown (View subdivision price trend),LT 12 BLK 1 MODERN MIDTOWN,...,NaN,NaN,NaN,$223.83,NaN,NaN,NaN,NaN,NaN,NaN
1,2,['https://photos.harstatic.com/190088153/hr/im...,"$ 450,000 ($223.33/sqft.) $Convert",118 Pierce Street,Houston,TX,77002,Harris County,Modern Midtown (View subdivision price trend),LT 8 BLK 1 MODERN MIDTOWN,...,NaN,NaN,NaN,$223.83,NaN,NaN,NaN,NaN,NaN,NaN
2,105,['https://photos.harstatic.com/190618667/hr/im...,"$ 575,000 ($232.98/sqft.) $Convert",1316 Hadley Street,Houston,TX,77002,Harris County,Austin Hadley Place,LT 4 BLK 1 AUSTIN HADLEY PLACE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,109,['https://photos.harstatic.com/191391009/hr/im...,"$ 514,900 ($208.88/sqft.) $Convert",2751 Freund Street,Houston,TX,77003,Harris County,EAST END ON THE BAYOU (View subdivision price...,Lot 9 of EAST END ON THE BAYOU SEC 3,...,"['8 x 5, 3rd', '8 , 5, 3rd']",NaN,NaN,$196.51,NaN,NaN,NaN,NaN,"['5 x 9, 3rd', '5 x 8, 4th', '5 x 8, 1st', '5 ...",NaN
4,110,['https://photos.harstatic.com/185138271/hr/im...,"$ 514,900 ($208.88/sqft.) $Convert",2765 Freund Street,Houston,TX,77003,Harris County,EAST END ON THE BAYOU (View subdivision price...,Lot 15 of EAST END ON THE BAYOU SEC 3,...,"['8 x 5, 3rd', '8 , 5, 3rd']",NaN,NaN,$196.51,NaN,NaN,NaN,NaN,"['5 x 9, 3rd', '5 x 8, 4th', '5 x 8, 1st', '5 ...",NaN


In our dataset `State` and `Property Type` are the same for all houses so, we can remove them:

In [7]:
single_family_df.drop(['Unnamed: 0','State:','Property Type:'],axis=1,inplace=True)

## 2.6 Missing Values<a id='2.6_Missing_Values'></a>

In [8]:
# function to find missing value and returning count abd %
def missing_cal(df):
    """This function calculates missing value 
    for datafaram passed in as parameter"""
    missing = pd.concat([single_family_df.isnull().sum(), 100 * single_family_df.isnull().mean()], axis=1)
    missing.columns=['count', '%']
    missing.sort_values(by='count',ascending=False)
    return missing

In [9]:
missing = missing_cal(single_family_df)
missing

,count,%
image_link,0,0.000000
Listing Price:,3,0.027189
Address:,0,0.000000
City:,0,0.000000
Zip Code:,0,0.000000
...,...,...
Utility Room Desc:,7144,64.745333
Sunroom:,10805,97.924597
Guest Suite:,10902,98.803698
Bath:,9361,84.837774


## 1.6.1 Features With >90% Missing Values<a id='1.6.1_Features_With_>90%_Missing_Values'></a>

Let's take a look at features with more than 90% missing values: 

In [10]:
missing = missing_cal(single_family_df)
nan_90 = missing.loc[missing['%']>90].index
print('Number of Features with more than 90% None: ',len(nan_90))

Number of Features with more than 90% None:  9


In [11]:
missing.loc[nan_90].sort_values(by="%")

,count,%
Extra Room:,9980,90.447707
Median Appraised Value / Square ft.:,10056,91.136487
Media Room:,10075,91.308682
Carport Description:,10446,94.671017
Water Amenity:,10609,96.148269
Garage Apartment:,10732,97.263005
Sunroom:,10805,97.924597
Wine Room:,10897,98.758383
Guest Suite:,10902,98.803698


We need to see what kind of information are in each of these features:

In [12]:
for item in nan_90:
    print('Value Count for '+item)
    print(single_family_df[item].value_counts())
    print('-'*100)

Value Count for Media Room:
['2nd', '2nd']                      30
['13x15, 2nd', '3.96 x 4.57(m)']    16
['14x13, 2nd', '4.27 x 3.96(m)']    15
['16x12, 2nd', '4.88 x 3.66(m)']    15
['15x13, 2nd', '4.57 x 3.96(m)']    14
                                    ..
['18X10, 2nd', '5.49 x 3.05(m)']     1
['17 x 12, 2nd', '17 , 12, 2nd']     1
['12 x 17, 2nd', '12 , 17, 2nd']     1
['17x12, 3rd', '5.18 x 3.66(m)']     1
['17X15, 1st', '5.18 x 4.57(m)']     1
Name: Media Room:, Length: 463, dtype: int64
----------------------------------------------------------------------------------------------------
Value Count for Water Amenity:
Lake View                                                     127
Lake View, Lakefront                                           99
Lakefront                                                      56
Pond                                                           55
Bayou View                                                     10
Bayou Frontage, Bayou View          

* Values for `Media Room`, `Extra Room`, `Wine Room`, `Sunroom`, `Guest Suite`and `Garage Apartment` are kind of dimension of each of those rooms along with some nonsense values like (`Yes` for `Garage Apartment`). 
* For `Water Amenity` there are to much unique categories and there is no way to be able to fill rest of none values with correct category
* `Carport Description` has 3 different categories for total 611 house and the rest do not have any carport so I will fill  all none values with new category as 'Not Applicable'.
* `Median Appraised Value / Square ft.:` is the fact (based on active listing) for each subdivision and can be fill by the value for same subdivision.

In [13]:
# Replacing None value for 'Carport Description:' with 'not applicable'
single_family_df['Carport Description:'][single_family_df['Carport Description:'].isnull()]='not applicable'

# Dropping 'Media Room:', 'Water Amenity:', 'Extra Room:', 'Wine Room:', 
#'Sunroom:', 'Guest Suite:', 'Garage Apartment:', 'Vacation Rental:'
single_family_df.drop(['Media Room:', 'Water Amenity:', 'Extra Room:', 'Wine Room:', 'Sunroom:', 'Guest Suite:', 
                       'Garage Apartment:'],axis=1,inplace=True)

## 1.6.2 Features With 80%-90% Missing Values<a id='1.6.2_Features_With_80%_-_90%_Missing_Values'></a>

Next step is looking at the features with more than 80% none values:

In [14]:
missing = missing_cal(single_family_df)
nan_80 = missing.loc[missing['%']>80].index
print('Number of Features with more than 80% None: ',len(nan_80))

Number of Features with more than 80% None:  14


In [15]:
missing.loc[nan_80].sort_values(by="%")

,count,%
Average Square Ft.:,9281,84.112742
Average Price/Square Ft.:,9281,84.112742
Market Area Name:,9282,84.121805
Home For Sales:,9282,84.121805
Average List Price:,9282,84.121805
Home For Lease:,9282,84.121805
Average Lease/Square Ft.:,9282,84.121805
Average Lease:,9283,84.130868
Bath:,9361,84.837774
Den:,9472,85.843756


In [16]:
#printing value count for each feature with more than 80 none value
for item in nan_80:
    print('Value Count for '+item)
    print(single_family_df[item].value_counts())
    print('-'*100)

Value Count for Private Pool Desc:
In Ground                                    357
Gunite, In Ground                            328
Gunite, Heated, In Ground                    229
Gunite                                       200
Heated, In Ground                            103
Gunite, Heated, In Ground, Salt Water         45
Gunite, Heated                                42
Above Ground                                  30
Heated, In Ground, Salt Water                 25
Gunite, In Ground, Salt Water                 18
In Ground, Salt Water                         17
Gunite, Salt Water                            10
Gunite, Heated, Salt Water                     8
Enclosed, Heated, In Ground                    6
In Ground, Vinyl Lined                         5
Heated                                         4
Heated, Salt Water                             3
Fiberglass                                     3
Salt Water                                     2
Enclosed, Gunite, In Ground       

* `Controlled Access`categories are mixed  of 'Automatic', 'Driveway', 'Manned' and 'Intercom' that makes me believe the rest of the house do not have any type of controlled access. I think filling none values with 'No controlled access' would be reasonable.
* Same as `Water Amenity` there are so many categories for 'Private Pool Desc'. After counting each category for `Private Pool:` groups figured out that there are description for house without private pool and I think it may happened by mistake and I decided to drop this column.
* `Master Planned Community` and `Market Area Name` categories seems to be same as subdivision name and we will deal with them later on subdivision section
* `Home For Sales`, `Average List Price`,`Average Square Ft.`,`Average Price/Square Ft.`, `Home For Lease`, `Average Lease`and `Average Lease/Square Ft.`, are the facts (based on active listing) for each subdivision and can be fill by the value for same subdivision.
* `Den` and `Bath` are dimension along with other values like '1th' which I think is typo mistake and I decided to drop them.

In [17]:
#counting 'Private Pool Desc:' category for `Private Pool:` groups
single_family_df.groupby('Private Pool:')['Private Pool Desc:'].value_counts()

Private Pool:  Private Pool Desc:                       
No             In Ground                                     12
               Above Ground                                   5
               Enclosed, Heated, In Ground                    5
               Heated, In Ground                              4
               Gunite                                         3
               Gunite, In Ground                              2
               Fiberglass                                     1
               Gunite, Heated, In Ground                      1
               Gunite, Heated, In Ground, Salt Water          1
Yes            In Ground                                    345
               Gunite, In Ground                            326
               Gunite, Heated, In Ground                    228
               Gunite                                       197
               Heated, In Ground                             99
               Gunite, Heated, In Ground, Salt 

In [18]:
single_family_df.drop(['Private Pool Desc:','Bath:','Den:'],axis=1,inplace=True)
# Replacing None value for 'Carport Description:' with 'not applicable'
single_family_df['Controlled Access:'][single_family_df['Controlled Access:'].isnull()]='no controlled access'

## 1.6.3 Features With 70%-80% Missing Values<a id='1.6.3_Features_With_70%_-_80%_Missing_Values'></a>

Now I investigating features with more than 70% none values:

In [19]:
missing = missing_cal(single_family_df)
nan_70 = missing.loc[((missing['%']>70 )& (missing['%']<80))].index
print('Number of Features with more than 70% None: ',len(nan_70))

Number of Features with more than 70% None:  2


In [20]:
missing.loc[nan_70].sort_values(by="%")

,count,%
Family Room:,7729,70.047127
Primary Bath:,8276,75.004531


In [21]:
#printing value count for each feature with more than 70 none value
for item in nan_70:
    print('Value Count for '+item)
    print(single_family_df[item].value_counts())
    print('-'*100)

Value Count for Family Room:
['1st', '1st']                              78
['18x16, 1st', '5.49 x 4.88(m)']            46
['20x17, 1st', '6.10 x 5.18(m)']            44
['21x17, 1st', '6.40 x 5.18(m)']            42
['19x18, 1st', '5.79 x 5.49(m)']            40
                                            ..
['20x14, 2nd', '6.10 x 4.27(m)']             1
['23\'8"x20\', 1st', '23\'8",20\', 1st']     1
["17'10X16'8, 1st", "17'10,16'8, 1st"]       1
['15X17, 1st', '4.57 x 5.18(m)']             1
['14.6 x 20, 1st', '14.6 , 20, 1st']         1
Name: Family Room:, Length: 919, dtype: int64
----------------------------------------------------------------------------------------------------
Value Count for Primary Bath:
['1st', '1st']                                        742
['2nd', '2nd']                                        304
['3rd', '3rd']                                        109
['12x10, 1st', '3.66 x 3.05(m)']                       25
['10x8, 1st', '3.05 x 2.44(m)']              

`Family Room` and `Primary Bath` are dimension for family room and master bath room and all houses should have these values and can not be 0. I think dropping these features would be appropriate since I can not fill values for more than 70% of houses.

In [22]:
single_family_df.drop(['Family Room:','Primary Bath:'],axis=1,inplace=True)

## 1.6.4 Features With 50%-70% Missing Values<a id='1.6.4_Features_With_50%_-_70%_Missing_Values'></a>

Next step is to look at features with none values between 50% and 70%:

In [23]:
missing = missing_cal(single_family_df)
nan_50_70 = missing.loc[((missing['%']>50 )& (missing['%']<70))].index
print('Number of Features with more than 50% and less than 70% None: ',len(nan_50_70))

Number of Features with more than 50% and less than 70% None:  6


In [24]:
missing.loc[nan_50_70].sort_values(by="%")

,count,%
Front Door:,6461,58.555374
Breakfast:,6651,60.277325
Garage Carport:,6675,60.494834
Utility Room Desc:,7144,64.745333
Game Room:,7224,65.470364
Study/Library:,7588,68.769259


In [25]:
#printing value count for each feature with more than 50 none value
for item in nan_50_70:
    print('Value Count for '+item)
    print(single_family_df[item].value_counts())
    print('-'*100)

Value Count for Breakfast:
['1st', '1st']                                      188
['11x10, 1st', '3.35 x 3.05(m)']                    155
['10x10, 1st', '3.05 x 3.05(m)']                    142
['12x10, 1st', '3.66 x 3.05(m)']                    135
['10x9, 1st', '3.05 x 2.74(m)']                     103
                                                   ... 
['11 x 17, 1st', '11 , 17, 1st']                      1
['8 X 6, 1st', '8 , 6, 1st']                          1
['12\'6"x18\'4.5", 1st', '12\'6",18\'4.5", 1st']      1
['23x12, 2nd', '7.01 x 3.66(m)']                      1
['9x6', '2.74 x 1.83(m)']                             1
Name: Breakfast:, Length: 730, dtype: int64
----------------------------------------------------------------------------------------------------
Value Count for Game Room:
['2nd', '2nd']                              67
['18x14, 2nd', '5.49 x 4.27(m)']            49
['19x16, 2nd', '5.79 x 4.88(m)']            46
['19x15, 2nd', '5.79 x 4.57(m)']            

We can calculate area for `Utility Room`, `Study/Library`. `Game Room`and `Breakfast` based on  dimension we have and assume the NA values are zero for those houses that do not have these rooms. I drop `Garage Carport` and `Front Door` since there is no information for rest of the house.

In [26]:
def area_calc(item,pattern = "([\d.]+)(?:.*?([\d]+))?.*?[x|*].*?([\d]+)(?:.*?([\d.]+))?"):
    """This function calculates are based on dimension passed in as parameter. Regular experession
    pattern will find dimension and group them as feet and inch"""
    pattern = re.compile(pattern,re.IGNORECASE)
    area=0
    dim=[]
    if type(item)==list:
        for i in item:
            if (('x' in i or 'X' in i or '*' in i )and '(m)' not in i ):
                dim.append(i.replace('[','').strip())
        for d in dim:
            d=d.replace(' ','').strip()
            match=pattern.findall(d)
            try:
                dimension_list = [float(item) if len(item)>0 else 0 for item in match[0]]
                area += (dimension_list[0]+(dimension_list[1]/12))*(dimension_list[2]+(dimension_list[3]/12))
            except:
                area=None
        return(area)

In [27]:
# Calculating area for Utility room
single_family_df['UtilitySqft'] = single_family_df['Utility Room Desc:'].str.split(',')
single_family_df['UtilitySqft'] = single_family_df['UtilitySqft'].apply(area_calc)

# Calculating area for Study/Library room
single_family_df['StudySqft'] = single_family_df['Study/Library:'].str.split(',')
single_family_df['StudySqft'] = single_family_df['StudySqft'].apply(area_calc)

# Calculating area for Game room
single_family_df['GameSqft'] = single_family_df['Game Room:'].str.split(',')
single_family_df['GameSqft'] = single_family_df['GameSqft'].apply(area_calc)

# Calculating area for Breakfast
single_family_df['BreakfastSqft'] = single_family_df['Breakfast:'].str.split(',')
single_family_df['BreakfastSqft'] = single_family_df['BreakfastSqft'].apply(area_calc)

single_family_df.update(single_family_df[['UtilitySqft','StudySqft','GameSqft','BreakfastSqft']].fillna(0))

In [28]:
single_family_df.drop(list(nan_50_70),axis=1,inplace=True)

So far I investigated features with the none value more than 50% and still need to dig more and also fill values for features that are the facts (based on active listing) for each subdivision like: `Home For Sales`, `Average List Price`,`Average Square Ft.`,`Average Price/Square Ft.`, `Home For Lease`, `Average Lease`and `Average Lease/Square Ft.`. But before that lets take a look at features with none values more than 10%:

## 1.6.5 Features With 10%-50% Missing Values<a id='1.6.5_Features_With_10%_-_50%_Missing_Values'></a>

In [29]:
missing = missing_cal(single_family_df)
nan_10_50 = missing.loc[((missing['%']>10 )& (missing['%']<50))].index
print('Number of Features with more than 10% and less than 50% None: ',len(nan_10_50))

Number of Features with more than 10% and less than 50% None:  35


In [30]:
missing.loc[nan_10_50].sort_values(by="%")

,count,%
Tax Rate:,1288,11.673011
Dishwasher:,1401,12.697118
Bedroom Desc:,1678,15.207540
Garage(s):,1759,15.941635
Median Appraised Value:,1769,16.032264
Median Year Built:,1769,16.032264
Median Lot Square Ft.:,1769,16.032264
Median Square Ft.:,1769,16.032264
Single Family Properties:,1769,16.032264
County / Zip Code:,1769,16.032264


In [31]:
#printing value count for each feature with more than 50 none value
for item in nan_10_50:
    print('Value Count for '+item)
    print(single_family_df[item].value_counts())
    print('-'*100)

Value Count for Garage(s):
2 / Attached              5211
2 / Detached               963
3 / Attached               703
1 / Detached               352
1 / Attached               287
                          ... 
21 / Attached                1
1 / Tandem                   1
20 / Attached                1
7 / Attached,Oversized       1
6 / Detached                 1
Name: Garage(s):, Length: 115, dtype: int64
----------------------------------------------------------------------------------------------------
Value Count for Living:
['1st', '1st']                      177
['18x15, 1st', '5.49 x 4.57(m)']     65
['15x13, 1st', '4.57 x 3.96(m)']     64
['14x12, 1st', '4.27 x 3.66(m)']     55
['15x12, 1st', '4.57 x 3.66(m)']     54
                                   ... 
['23 x 22, 1st', '23 , 22, 1st']      1
['19 x 16, 2nd', '19 , 16, 2nd']      1
['27x10, 1st', '8.23 x 3.05(m)']      1
['28x21, 1st', '8.53 x 6.40(m)']      1
['27X16, 2nd', '8.23 x 4.88(m)']      1
Name: Living:, Length: 

`Room Description`, `Countertop`, `Floors`, `Bedroom Desc`, `Kitchen Desc`, `Bathroom Description`, `Connections`, `Oven`, `Range`, `Energy Feature`, `Interior`, `Exterior`, `Financing Considered` are just information and we can not fill them with unknown values since some may not be accurate and I don't think they are relevant to our analysis so, I will drop all of them.
We also do not need `County / Zip Code:` since we do have another column for zip codes.
I could not find any information regarding to `Single Family Properties:` so, this column will be dropped as well.

In [32]:
single_family_df.drop(['Room Description:', 'Countertop:', 'Floors:', 'Bedroom Desc:', 'Kitchen Desc:', 
                       'Bathroom Description:','Connections:', 'Oven:', 'Range:', 'Energy Feature:',
                       'Interior:', 'Exterior:', 'Financing Considered:','Single Family Properties:','County / Zip Code:'], axis=1,inplace=True)

`Ice Maker`, `Microwave`, `Compactor`, `Dishwasher`, `Disposal` and `Area Pool` are 'Yes/No' categories and I think it is relevant to fill none values with 'No'. For sure it is a little bit optimistic since some houses may have those features and owner/agent forgot to fill them but for now filling with 'No' value is the best way to dealing with them.

In [33]:
single_family_df['Disposal:'][single_family_df['Disposal:'].isnull()]='No'
single_family_df['Ice Maker:'][single_family_df['Ice Maker:'].isnull()]='No'
single_family_df['Compactor:'][single_family_df['Compactor:'].isnull()]='No'
single_family_df['Area Pool:'][single_family_df['Area Pool:'].isnull()]='No'
single_family_df['Microwave:'][single_family_df['Microwave:'].isnull()]='No'
single_family_df['Dishwasher:'][single_family_df['Dishwasher:'].isnull()]='No'

At this point I am investigating other features indevisually:

## 1.6.5.1 Garage<a id='1.6.5.1_Garage'></a>

In [34]:
single_family_df['Garage(s):'].value_counts()

2 / Attached              5211
2 / Detached               963
3 / Attached               703
1 / Detached               352
1 / Attached               287
                          ... 
21 / Attached                1
1 / Tandem                   1
20 / Attached                1
7 / Attached,Oversized       1
6 / Detached                 1
Name: Garage(s):, Length: 115, dtype: int64

The important part of this feature is the number of garage each house has. Also we now almost every single family homes have at least 2 garages and it is relevant to fill none values with '2'.

In [35]:
single_family_df['Garage(s):'].fillna('2',inplace=True)
single_family_df['Garage'] = [item[0] if item !=None else 0 for item in single_family_df['Garage(s):'].str.split(' ') ]
single_family_df['Garage']=single_family_df['Garage'].astype(int)
single_family_df.drop('Garage(s):',axis=1,inplace=True)
single_family_df['Garage'].value_counts()

2     8620
3     1439
1      735
4      176
5       19
6       11
8        5
7        4
24       3
40       3
63       2
36       2
10       2
14       1
56       1
22       1
9        1
49       1
60       1
42       1
58       1
27       1
21       1
20       1
28       1
26       1
Name: Garage, dtype: int64

As you can see there are some houses with more than 10 garage which is odd. After checking images for some of these houses in www.HAR.com it seems those have only 2 garage and I fill those values with 2 which is the median of this feature.

In [36]:
single_family_df.Garage[single_family_df['Garage']>7]=single_family_df['Garage'].median()
single_family_df['Garage'].value_counts()

2    8650
3    1439
1     735
4     176
5      19
6      11
7       4
Name: Garage, dtype: int64

## 1.6.5.2 Living<a id='1.6.5.2_Living'></a>

To calculate the living area I use area calculation function, pass in the dimension of the living room and return the area and for the next step I will fill none values with the average of living room area per subdivision

In [37]:
single_family_df['LivingSqft'] = single_family_df['Living:'].str.split(',')
single_family_df['LivingSqft'] = single_family_df['LivingSqft'].apply(area_calc)        

In [38]:
single_family_df['LivingSqft'].describe()

count    6333.000000
mean      281.339938
std       158.494631
min         0.000000
25%       204.000000
50%       272.000000
75%       342.000000
max      6651.000000
Name: LivingSqft, dtype: float64

In [39]:
single_family_df[single_family_df.LivingSqft==0][['LivingSqft','Living:','Address:','Building Sqft.:']]

,LivingSqft,Living:,Address:,Building Sqft.:
20,0.0,"['2nd', '2nd']",619 Live Oak Street,"2,671248(mÂ²) /Appraisal District"
43,0.0,"['2nd', '2nd']",3805 Wilmer Street,"3,640338(mÂ²) /Appraisal District"
63,0.0,"['2nd', '2nd']",15 N Ennis Street,"1,812168(mÂ²) /Appraisal District"
148,0.0,"['2nd', '2nd']",1914 Blodgett Street C,"2,540236(mÂ²) /Appraisal District"
165,0.0,"['2nd', '2nd']",1716 Rosewood Street C,"2,763257(mÂ²) /Builder"
...,...,...,...,...
10674,0.0,"['1st', '1st']",2306 Sledge Street,84078(mÂ²) /Seller
10694,0.0,"['1st', '1st']",15918 Chimney Rock Road,"2,205205(mÂ²) /Appraisal District"
10759,0.0,"['1st', '1st']",6517 Manorwood Drive,"3,630337(mÂ²) /Appraisal District"
10834,0.0,"['1st', '1st']",4622 Shadow Grass Drive,"3,356312(mÂ²) /Appraisal District"


In [40]:
single_family_df.loc[single_family_df['LivingSqft'] == 0, 'LivingSqft'] = np.NAN

In [41]:
single_family_df[single_family_df.LivingSqft>700][['LivingSqft','Living:','Address:','Building Sqft.:']]

,LivingSqft,Living:,Address:,Building Sqft.:
227,750.000000,"['25x30, 1st', '7.62 x 9.14(m)']",3914 Purdue Street,"5,370499(mÂ²) /Builder"
270,750.000000,"['30x25, 1st', '9.14 x 7.62(m)']",2007 Dunstan Road,"3,675341(mÂ²) /Appraisal District"
301,918.000000,"['34x27, 1st', '10.36 x 8.23(m)']",2 Longfellow Lane,"12,8081,190(mÂ²) /Appraisal District"
617,840.000000,"['30 x 28, 2nd', '30 , 28, 2nd']",5341 Larkin Street,"2,379221(mÂ²) /Builder"
865,1696.333333,"['12 1/2 x 14, 1st', '12 1/2 , 14, 1st']",515 W 14th Street,"1,135105(mÂ²) /Appraisal District"
1350,872.000000,"['21x24, 1st', '16x23, 2nd', '6.40 x 7.32(m)',...",931 W 42nd Street,"4,842450(mÂ²) /Appraisal District"
1550,726.000000,"['22 x 33, 2nd', '22 , 33, 2nd']",2025 McDuffie Street,"7,405688(mÂ²) /Appraisal"
1648,900.000000,"['30x30, 1st', '9.14 x 9.14(m)']",9 West Lane,"7,551701(mÂ²) /Appraisal District"
2460,750.000000,"['25x30, 1st', '7.62 x 9.14(m)']",9519 W Airport Boulevard,"1,845171(mÂ²) /Appraisal District"
2483,1533.333333,"['15 3/4 x 10, 1st', '15 3/4 , 10, 1st']",5926 Southville Street,"1,478137(mÂ²) /Seller"


It seems there are some typo mistakes here because living area can not be more than total building Sqft. I am checking HAR.com to see which one has a wrong dimension for living area.

In [42]:
#replacing right area measured
single_family_df.loc[865,'LivingSqft'] = 175
single_family_df.loc[2483,'LivingSqft'] = 157.5
single_family_df.loc[2844,'LivingSqft'] = 363.4
single_family_df.loc[3383,'LivingSqft'] = 126.034
single_family_df.loc[4286,'LivingSqft'] = 286
single_family_df.loc[5796,'LivingSqft'] = 360
single_family_df.loc[7226,'LivingSqft'] = 360

## 1.6.5.3 Dining<a id='1.6.5.3_Dining'></a>

In [43]:
single_family_df['Dining:'].value_counts()

['12x11, 1st', '3.66 x 3.35(m)']            219
['13x11, 1st', '3.96 x 3.35(m)']            205
['1st', '1st']                              188
['14x12, 1st', '4.27 x 3.66(m)']            182
['13x12, 1st', '3.96 x 3.66(m)']            162
                                           ... 
['15.8 x 11.6, 1st', '15.8 , 11.6, 1st']      1
['11.5x21, 1st', '3.51 x 6.40(m)']            1
['8 x 10, 1st', '8 , 10, 1st']                1
['10x14, 2nd', '3.05 x 4.27(m)']              1
['12.8 x 11.6, 2nd', '12.8 , 11.6, 2nd']      1
Name: Dining:, Length: 1245, dtype: int64

In [44]:
single_family_df['DiningSqft'] = single_family_df['Dining:'].str.split(',')
single_family_df['DiningSqft'] = single_family_df['DiningSqft'].apply(area_calc) 

In [45]:
single_family_df[['Dining:','DiningSqft']].sample(20,random_state=101)

,Dining:,DiningSqft
4697,"['13x10, 1st', '3.96 x 3.05(m)']",130.0
385,"['11x15, 1st', '3.35 x 4.57(m)']",165.0
1250,NaN,NaN
9045,"['14 x 12, 1st', '14 , 12, 1st']",168.0
2246,NaN,NaN
3439,NaN,NaN
6780,"['17x11, 1st', '5.18 x 3.35(m)']",187.0
6537,NaN,NaN
10689,NaN,NaN
6509,NaN,NaN


In [46]:
single_family_df[single_family_df.DiningSqft==0][['DiningSqft','Dining:','Address:','Building Sqft.:']]

,DiningSqft,Dining:,Address:,Building Sqft.:
43,0.0,"['2nd', '2nd']",3805 Wilmer Street,"3,640338(mÂ²) /Appraisal District"
53,0.0,"['2nd', '2nd']",448 N Live Oak Street,"1,910177(mÂ²) /Appraisal"
299,0.0,"['1st', '1st']",2225 Bolsover Street,"4,546422(mÂ²) /Appraisal District"
495,0.0,"['2nd', '2nd']",1626 A Dunlavy st Street,"2,723253(mÂ²) /Appraisal District"
532,0.0,"['1st', '1st']",530 Oxford Street,"3,067285(mÂ²) /Appraisal District"
...,...,...,...,...
10704,0.0,"['1st', '1st']",2022 GLEN PARK Drive,"2,356219(mÂ²) /Appraisal District"
10744,0.0,"['1st', '1st']",7411 Palmetto Springs Trail,"5,646525(mÂ²) /Builder"
10759,0.0,"['1st', '1st']",6517 Manorwood Drive,"3,630337(mÂ²) /Appraisal District"
10840,0.0,"['1st', '1st']",2814 Crawford,"4,687435(mÂ²) /Appraisal District"


In [47]:
single_family_df.loc[single_family_df['DiningSqft'] == 0, 'DiningSqft'] = np.NAN

In [48]:
single_family_df[single_family_df.DiningSqft>500][['DiningSqft','Dining:','Address:','Building Sqft.:']]

,DiningSqft,Dining:,Address:,Building Sqft.:
865,12520.555556,"['12 1/2 x 10 3/4, 1st', '12 1/2 , 10 3/4, 1st']",515 W 14th Street,"1,135105(mÂ²) /Appraisal District"
2324,1200.000000,"['30x20, 1st', '30x20, 1st', '9.14 x 6.10(m)',...",3429 Ella Lee Lane,"4,741440(mÂ²) /Appraisal District"
3471,655.590000,"['23.4 x 14.10, 1st', '23.4 x 14.10, 1st', '23...",5113 Itiel Street,"1,498139(mÂ²) /Builder"
3693,504.000000,"['18x14, Formal, 1st,', '18x14, Formal, 1st,']",5360 Spring Park Street,"5,140478(mÂ²) /Appraisal District"
4957,528.000000,"['24X22, 1st', '7.32 x 6.71(m)']",9303 Bintliff Drive,"1,837171(mÂ²) /Appraisal District"
5103,1456.000000,"['13X112, 1st', '3.96 x 34.14(m)']",1319 Mission Chase Drive,"2,388222(mÂ²) /Appraisal District"
6825,1200.000000,"['12x100, 1st', '3.66 x 30.48(m)']",10723 Braewick Drive,"2,068192(mÂ²) /Appraisal District"
7326,504.000000,"['28x18, 1st', '8.53 x 5.49(m)']",10 Magnolia Woods Drive,"12,7081,181(mÂ²) /Appraisal District"
9566,576.000000,"['18x16, 1st', '18x16, 1st', '5.49 x 4.88(m)',...",13606 Leon Springs Lane,"4,833449(mÂ²) /Builder"
10681,1817.500000,"['12 1/2 X 15, 1st', '12 1/2 , 15, 1st']",29200 Clydesdale Drive,"2,700251(mÂ²) /Builder"


Same as living area there are some mistake for dimension which need to be fixed.

In [49]:
single_family_df.loc[865,'DiningSqft'] = 134.375
single_family_df.loc[3471,'DiningSqft'] = 329.94
single_family_df.loc[5103,'DiningSqft'] = 143
single_family_df.loc[6825,'DiningSqft'] = 120
single_family_df.loc[10681,'DiningSqft'] = 187.5
single_family_df.loc[10691,'DiningSqft'] = 165
single_family_df.loc[10911,'DiningSqft'] = 208

## 1.6.5.4 Kitchen<a id='1.6.5.4_Kitchen'></a>

I am using same function to calculate kitchen are in sqft.

In [50]:
single_family_df['KitchenSqft'] = single_family_df['Kitchen:'].str.split(',')
single_family_df['KitchenSqft'] = single_family_df['KitchenSqft'].apply(area_calc) 

In [51]:
single_family_df[['Kitchen:','KitchenSqft']].sample(20,random_state=101)

,Kitchen:,KitchenSqft
4697,"['10x10, 1st', '3.05 x 3.05(m)']",100.0
385,"['10x15, 1st', '3.05 x 4.57(m)']",150.0
1250,"['14x8, 1st', '4.27 x 2.44(m)']",112.0
9045,"['18 x 10, 1st', '18 , 10, 1st']",180.0
2246,NaN,NaN
3439,NaN,NaN
6780,"['12x11, 1st', '3.66 x 3.35(m)']",132.0
6537,"['15x13, 1st', '4.57 x 3.96(m)']",195.0
10689,NaN,NaN
6509,"['12x6, 1st', '3.66 x 1.83(m)']",72.0


In [52]:
single_family_df[single_family_df.KitchenSqft==0][['KitchenSqft','Kitchen:','Address:','Building Sqft.:']]

,KitchenSqft,Kitchen:,Address:,Building Sqft.:
20,0.0,"['2nd', '2nd']",619 Live Oak Street,"2,671248(mÂ²) /Appraisal District"
43,0.0,"['2nd', '2nd']",3805 Wilmer Street,"3,640338(mÂ²) /Appraisal District"
63,0.0,"['2nd', '2nd']",15 N Ennis Street,"1,812168(mÂ²) /Appraisal District"
299,0.0,"['1st', '1st']",2225 Bolsover Street,"4,546422(mÂ²) /Appraisal District"
345,0.0,"['1st', '1st']",6004 Buffalo Speedway,"2,106196(mÂ²) /Appraisal District"
...,...,...,...,...
10875,0.0,"['1st', '1st']",23606 Sitka Spruce Drive,"3,577332(mÂ²) /Builder"
10877,0.0,"['1st', '1st']",23826 Padauk Tree Trail,"3,577332(mÂ²) /Builder"
10906,0.0,"['1st', '1st']",1815 Rice Mill Drive,"4,058377(mÂ²) /Appraisal District"
10930,0.0,"['1st', '1st']",6343 Winthrop Terrace Trail,"2,541236(mÂ²) /Builder"


In [53]:
single_family_df.loc[single_family_df['KitchenSqft'] == 0, 'KitchenSqft'] = np.NAN

In [54]:
single_family_df[single_family_df.KitchenSqft>500][['KitchenSqft','Kitchen:','Address:','Building Sqft.:']]

,KitchenSqft,Kitchen:,Address:,Building Sqft.:
109,638.000000,"['22x29, 1st', '6.71 x 8.84(m)']",4811 Jackson Street,"4,417410(mÂ²) /Builder"
301,1161.000000,"['43x27, 1st', '13.11 x 8.23(m)']",2 Longfellow Lane,"12,8081,190(mÂ²) /Appraisal District"
304,516.000000,"['21.5 x 24, 1st', '21.5 , 24, 1st']",6335 Belmont Street,"7,275676(mÂ²) /Builder"
311,720.000000,"['36 x 20, 1st', '36 , 20, 1st']",2616 Quenby Avenue,"5,775536(mÂ²) /Appraisal District"
730,3225.000000,"['15x215, 2nd', '4.57 x 65.53(m)']",1115 Herkimer Street,"2,293213(mÂ²) /Appraisal District"
865,1575.166667,"['12 1/2 x 13, 1st', '12 1/2 , 13, 1st']",515 W 14th Street,"1,135105(mÂ²) /Appraisal District"
1014,532.000000,"['28x19, 2nd', '8.53 x 5.79(m)']",713 Booth Street,"5,582519(mÂ²) /Appraisal District"
1350,509.000000,"['19x22, 1st', '7x13, 2nd', '5.79 x 6.71(m)', ...",931 W 42nd Street,"4,842450(mÂ²) /Appraisal District"
2123,600.000000,"['24X25, 1st', '7.32 x 7.62(m)']",3726 N Braeswood Boulevard,"3,800353(mÂ²) /Appraisal District"
2174,600.000000,"['30 X 20, 1st', '30 , 20, 1st']",4059 Grennoch Lane,"5,571518(mÂ²) /Appraisal District"


In [55]:
single_family_df.loc[730,'KitchenSqft'] = 322.5
single_family_df.loc[865,'KitchenSqft'] = 162.5
single_family_df.loc[2483,'KitchenSqft'] = 75
single_family_df.loc[2768,'KitchenSqft'] = 84
single_family_df.loc[4702,'KitchenSqft'] = 140
single_family_df.loc[7757,'KitchenSqft'] = 330
single_family_df.loc[7662,'KitchenSqft'] = 156.8
single_family_df.loc[8224,'KitchenSqft'] = 252
single_family_df.loc[8278,'KitchenSqft'] = 304
single_family_df.loc[9099,'KitchenSqft'] = 203
single_family_df.loc[10483,'KitchenSqft'] = 255
single_family_df.loc[10658,'KitchenSqft'] = 280
single_family_df.loc[10681,'KitchenSqft'] = 175

Now we can drop old living, dining and kitchen columns:

In [56]:
single_family_df.drop(['Living:', 'Dining:', 'Kitchen:'], axis=1,inplace=True)

## 1.7 Subdivisions And Their Facts<a id='1.7_Subdivisions_And_their_Facts'></a>

To uniform subdivision I scraped all subdivision names from HAR.com and will replace names with correct one based on similarity:

In [57]:
sub_df = pd.read_csv('../data/raw/Subdivision.csv')
sub_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [58]:
sub_df.head()

,Subdivision,Zip,Med.Appraisal,Avg.Sqft.,Avg.Yr.Built
0,MARLOWE CONDOS,77002,"$522,701","1,100",2018.0
1,Modern Midtown,77002,"$469,147","2,096",2014.0
2,Midtowne Plaza,77002,"$439,282","2,507",1999.0
3,Macgregor Demerritt,77002,"$438,234","2,034",1930.0
4,Hermann Lofts Condo,77002,"$385,446","1,546",1998.0


In [59]:
sub_df.Subdivision.unique()

array(['MARLOWE CONDOS', 'Modern Midtown', 'Midtowne Plaza', ...,
       'Pecan Terrace', 'Merilyn Plaza', 'Allen Genoa Condo'],
      dtype=object)

As above table shown Med.Appraisal, Avg.Sqft. and Avg.Yr.Built are same for each subdivision and we can fill none values with these numbers for each column.

In [60]:
single_family_df[['Subdivision:','Subdivision Name:','Market Area Name:','Master Planned Community:','Zip Code:']].head()

,Subdivision:,Subdivision Name:,Market Area Name:,Master Planned Community:,Zip Code:
0,Modern Midtown (View subdivision price trend),Modern Midtown,NaN,NaN,77002
1,Modern Midtown (View subdivision price trend),Modern Midtown,NaN,NaN,77002
2,Austin Hadley Place,NaN,Midtown - Houston,NaN,77002
3,EAST END ON THE BAYOU (View subdivision price...,East End On The Bayou,NaN,NaN,77003
4,EAST END ON THE BAYOU (View subdivision price...,East End On The Bayou,NaN,NaN,77003


In [61]:
single_family_df[['Subdivision:','Subdivision Name:','Market Area Name:','Master Planned Community:']].isna().sum()

Subdivision:                    4
Subdivision Name:            1769
Market Area Name:            9282
Master Planned Community:    9626
dtype: int64

It seems all these 4 columns are the same. As we can see `Subdivision Name` has more clean name so, I will keep only this column and fill null values with market area name and then use subdivision datafram to change all names to standard one and then drop the rest.

In [62]:
single_family_df['Subdivision Name:'] = single_family_df['Subdivision Name:'].fillna(single_family_df['Market Area Name:'])

In [63]:
single_family_df['Subdivision Name:'].isna().sum()

17

In [64]:
single_family_df[single_family_df['Subdivision Name:'].isna()][['Subdivision:','Year Built:','Subdivision Name:','Market Area Name:','Master Planned Community:']]

,Subdivision:,Year Built:,Subdivision Name:,Market Area Name:,Master Planned Community:
73,Mckinney Lndg Sub (View subdivision price trend),2013 / Appraisal District,NaN,NaN,NaN
491,VERMONT STREET GROVE,2020 / Builder,NaN,NaN,NaN
509,Magnolia Grove (View subdivision price trend),1950 / Appraisal District,NaN,NaN,NaN
853,Heights Homes/Herkimer Sub (View subdivision ...,2013 / Appraisal District,NaN,NaN,NaN
1431,Shepherd Oaks (View subdivision price trend),2018 / Appraisal District,NaN,NaN,NaN
2002,Oaks of Lawndale,2020 / Builder,NaN,NaN,NaN
3005,Riverview Place (View subdivision price trend),2019 / Builder,NaN,NaN,NaN
3077,Lakeside T/H (View subdivision price trend),1984 / Appraisal District,NaN,NaN,NaN
3080,Lakeside T/H (View subdivision price trend),1977 / Appraisal District,NaN,NaN,NaN
3081,Lakeside T/H (View subdivision price trend),1982 / Appraisal District,NaN,NaN,NaN


most of these null values for subdivision are new but still we can fill them by values from Subdivision:

In [65]:
single_family_df['Subdivision Name:'] = single_family_df['Subdivision Name:'].fillna(single_family_df['Subdivision:'])

In [66]:
single_family_df['SubName'] = single_family_df['Subdivision Name:']

In [67]:
single_family_df['SubName'].isna().sum()

0

In [68]:
single_family_df.drop(['Subdivision Name:','Subdivision:','Market Area Name:','Master Planned Community:'],
                      axis=1,inplace=True)

In [ ]:
standard_subdivision=list(sub_df.Subdivision.unique())
for sub in tqdm(standard_subdivision):
    matches = process.extract(sub, single_family_df.SubName,limit = 1000)
        
# For each possible_match with similarity score >= 90
    for possible_match in matches:
        if possible_match[1] >= 90:
            matching = single_family_df.SubName == possible_match[0]
        else:
            continue
        single_family_df.loc[matching , 'SubName'] = sub
print(single_family_df.SubName.unique())

 23%|█████████████████▎                                                           | 900/3995 [21:15<1:01:35,  1.19s/it]

Now we can take a look at those features related to each subdivision like: `Median Appraised Value:`, `Median Year Built:`, 
`Median Lot Square Ft.:`, `Median Square Ft.:`, `Neighborhood Value Range:`.	

In [ ]:
single_family_df[['Median Appraised Value:', 'Median Year Built:', 'Median Lot Square Ft.:', 'Median Square Ft.:', 'Neighborhood Value Range:']].isna().sum()

Those features have same number of missing values and it seems it may because of value missing for some the subdivisions. lets take a look:

In [ ]:
single_family_df.SubName.loc[single_family_df['Median Appraised Value:'].isna()].value_counts()

In [ ]:
sub_df.loc[sub_df.Subdivision=='cane island']

In [ ]:
single_family_df['Year Built:'].loc[single_family_df['Median Lot Square Ft.:'].isna()].value_counts()

It seems there is no information for those subdivisions in HAR.com since more than 90% of them are for new subdivisions. so I will drop those rows.

In [ ]:
single_family_df=single_family_df[~single_family_df['Median Appraised Value:'].isnull()]

In [ ]:
single_family_df[['Median Appraised Value:', 'Median Year Built:', 'Median Lot Square Ft.:', 'Median Square Ft.:', 'Neighborhood Value Range:']]

Now we need to clean those features and change their type to the right one.

In [ ]:
single_family_df['MedianApp'] = single_family_df['Median Appraised Value:'].str.replace('$','').str.replace(',','').str.strip()
single_family_df['MedianApp'] = pd.to_numeric(single_family_df['MedianApp'],errors='coerce')

In [ ]:
single_family_df['MedianApp']

In [ ]:
single_family_df['MedianYearBlt'] = pd.to_datetime(single_family_df['Median Year Built:'],format='%Y').dt.year

In [ ]:
single_family_df['MedianYearBlt']

In [ ]:
single_family_df['MedianSqft'] = single_family_df['Median Lot Square Ft.:'].str.replace(',','').str.strip()
single_family_df['MedianSqft'] = pd.to_numeric(single_family_df['MedianSqft'],errors='coerce')

In [ ]:
single_family_df['MedianSqft'] 

In [ ]:
single_family_df['NeighborValRangeMin'] = single_family_df['Neighborhood Value Range:'].apply(lambda x:x.split('-')[0].strip().replace('$',''))
single_family_df['NeighborValRangeMin'] = pd.to_numeric(single_family_df['NeighborValRangeMin'],errors='coerce')

In [ ]:
single_family_df['NeighborValRangeMin'] 

In [ ]:
single_family_df['NeighborValRangeMax'] = single_family_df['Neighborhood Value Range:'].apply(lambda x:x.split('-')[1].replace('$','').replace('K','').strip())
single_family_df['NeighborValRangeMax'] = pd.to_numeric(single_family_df['NeighborValRangeMax'],errors='coerce')

In [ ]:
single_family_df['NeighborValRangeMax'] 

Instead of having 2 min and max columns for neighborhood value I think using average of these two range would be better.

In [ ]:
single_family_df['AvgNeighborValRange'] = ((single_family_df.NeighborValRangeMax + single_family_df.NeighborValRangeMin)/2)*1000

In [ ]:
single_family_df.drop(['Median Appraised Value:','NeighborValRangeMax', 'NeighborValRangeMin','Median Year Built:', 'Median Lot Square Ft.:', 'Median Square Ft.:', 'Neighborhood Value Range:'],
                      axis=1,inplace=True)

In [ ]:
single_family_df.info()

We have some features with 100% null values. lets drop them:

In [ ]:
single_family_df.drop(['Home For Sales:', 'Average List Price:', 'Average Square Ft.:', 'Average Price/Square Ft.:',
                       'Home For Lease:','Average Lease:','Average Lease/Square Ft.:'],axis=1,inplace=True)

Now we are checking missing values again:

In [ ]:
missing = missing_cal(single_family_df)
missing.loc[missing['%'] > 0].sort_values(by="%",ascending=False)

In [ ]:
single_family_df.groupby('SubName').count()['Taxes w/o Exemp:']

In [ ]:
single_family_df[['Taxes w/o Exemp:','Listing Price:','Median Price / Square ft.:','Tax Rate:','SubName']]

Tax rate and Median Price / Square ft. would be the same for all house in same subdivision and Taxes paid is different for each house based on tax rate, total house sqft and other factor. I think filling null value for tax paid with the average of taxes per subdivision would be appropriate but first we need to clean these two columns.

In [ ]:
single_family_df['MedianPrice/Sqft'] = pd.to_numeric(single_family_df['Median Price / Square ft.:'].str
                                                     .replace('$','').str.strip(),errors='coerce')

single_family_df['PaidTax'] = pd.to_numeric(single_family_df['Taxes w/o Exemp:'].apply(lambda x:x.split('/')[0].replace(',','')
                                                                         .replace(' ','').strip().replace('$','') if 
                                                                         not pd.isna(x) else None),errors='coerce')
single_family_df['TaxRate'] = pd.to_numeric(single_family_df['Tax Rate:'],errors='coerce')

In [ ]:
single_family_df['PaidTax'] = single_family_df.groupby('SubName')['PaidTax'].transform(lambda x: x.fillna(x.mean()))
single_family_df['TaxRate'] = single_family_df.groupby('SubName')['TaxRate'].transform(lambda x: x.fillna(x.mean()))
single_family_df['MedianPrice/Sqft'] = single_family_df.groupby('SubName')['MedianPrice/Sqft'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
single_family_df['MedianPrice/Sqft'].isna().sum()

We still have some null values and it seems there is no information for those subdivisions. lets drop them.

In [ ]:
single_family_df=single_family_df[~single_family_df['MedianPrice/Sqft'].isnull()]

In [ ]:
single_family_df['TaxRate'].isna().sum()

In [ ]:
single_family_df=single_family_df[~single_family_df['TaxRate'].isnull()]

In [ ]:
single_family_df['PaidTax'].isna().sum()

In [ ]:
single_family_df=single_family_df[~single_family_df['PaidTax'].isnull()]

In [ ]:
single_family_df.drop(['Taxes w/o Exemp:','Tax Rate:','Median Price / Square ft.:'],axis=1,inplace=True)

In [ ]:
single_family_df[single_family_df.TaxRate>10]

In [ ]:
single_family_df.SubName[single_family_df.TaxRate>10]

In [ ]:
single_family_df.TaxRate[single_family_df.SubName=='Avalon Place']

In [ ]:
single_family_df.TaxRate[single_family_df.SubName=='Ayrshire']

In [ ]:
single_family_df.TaxRate[single_family_df.SubName=='Windermere Lakes']

In [ ]:
single_family_df.TaxRate[single_family_df.SubName=='Estates At Cullen Park']

In [ ]:
single_family_df['TaxRate'] = np.where(single_family_df.SubName=='Avalon Place',2.4216,single_family_df['TaxRate'])
single_family_df['TaxRate'] = np.where(single_family_df.SubName=='Ayrshire',2.4216,single_family_df['TaxRate'])
single_family_df['TaxRate'] = np.where(single_family_df.SubName=='Windermere Lakes',2.3243,single_family_df['TaxRate'])
single_family_df['TaxRate'] = np.where(single_family_df.SubName=='Estates At Cullen Park',2.7748,single_family_df['TaxRate'])

These 4 houses have tax rate which is out of range, I will replace it with the correct tax rate for its subdivision(avalon place and ayrshire)

In [ ]:
single_family_df.TaxRate[single_family_df.SubName=='Ayrshire']

In [ ]:
single_family_df.PaidTax.describe()

Lets take a look at maximum tax paid to see why it is too high.

In [ ]:
single_family_df[single_family_df.PaidTax > 100000][['Listing Price:','Address:','PaidTax','SubName','TaxRate']]

18 houses paid more than \\$100,000 for tax and most of them have price more that 3 million dollars but there is 2 houses in `Memorial Plaza` and `pinelakes pinaster pointe` subdivisions which paid more than normal for tax which should not be right. The house price is \\$739,000 and \\$329,000 tax rate for that area is 2.5 and 2.8 so the tax should not be that high.

In [ ]:
single_family_df.loc[5449,'PaidTax'] = 15400
single_family_df.loc[8725,'PaidTax'] = 8553

In [ ]:
single_family_df = single_family_df[single_family_df.PaidTax < 200000]

## 1.8 Fill Null Kitchen, Dining and Living <a id='1.8_Fill_Null_For_Kitchen_Dining_Living'></a>

I think we can fill null values for living, dining and kitchen area with them median sqft for each subdivision since most of the house in one subdivision are almost the same. To be more accurate I use median instead of mean in case if there is any outliers.

In [ ]:
SubGrouped = single_family_df.groupby('SubName')

In [ ]:
print('Average Living area for Tanglewood Trail is :',SubGrouped.LivingSqft.get_group('Tanglewood Trail').mean())
print('Median Living area for Tanglewood Trail is :',SubGrouped.LivingSqft.get_group('Tanglewood Trail').median())
SubGrouped.LivingSqft.get_group('Tanglewood Trail').hist()

In [ ]:
SubGrouped.DiningSqft.get_group('Tanglewood Trail').hist(bins=10)

In [ ]:
SubGrouped.KitchenSqft.get_group('Tanglewood Trail').hist(bins=10)


In [ ]:
single_family_df['LivingSqft'] = single_family_df.groupby('SubName')['LivingSqft'].apply(lambda x: x.fillna(x.median()))
single_family_df['DiningSqft'] = single_family_df.groupby('SubName')['DiningSqft'].apply(lambda x: x.fillna(x.median()))
single_family_df['KitchenSqft'] = single_family_df.groupby('SubName')['KitchenSqft'].apply(lambda x: x.fillna(x.median()))

In [ ]:
single_family_df['LivingSqft'].isna().sum()

In [ ]:
single_family_df['DiningSqft'].isna().sum()

In [ ]:
single_family_df['KitchenSqft'].isna().sum()

We still have some null values and it seems there is no information for those subdivisions. lets drop them.

In [ ]:
single_family_df=single_family_df[~single_family_df['DiningSqft'].isnull()]

In [ ]:
single_family_df['LivingSqft'].isna().sum()

In [ ]:
single_family_df['KitchenSqft'].isna().sum()

In [ ]:
single_family_df=single_family_df[~single_family_df['LivingSqft'].isnull()]

In [ ]:
single_family_df['KitchenSqft'].isna().sum()

In [ ]:
single_family_df=single_family_df[~single_family_df['KitchenSqft'].isnull()]

## 1.9 Listing Price<a id='1.9_Listing_Price'></a>

In [ ]:
single_family_df['Listing Price:'].isna().sum()

In [ ]:
single_family_df=single_family_df[~single_family_df['Listing Price:'].isnull()]

In [ ]:
single_family_df['Listing Price:']=single_family_df['Listing Price:'].str.split(' ').str[1]
single_family_df['Listing Price:']=single_family_df['Listing Price:'].str.replace(',','')
single_family_df['Listing Price:']=pd.to_numeric(single_family_df['Listing Price:'])
single_family_df.rename(columns = {'Listing Price:':'ListingPrice','Address:':'Address', 'Zip Code:':'ZipCode', 'County:':'County',
                                 'Subdivision:':'sub', 'Legal Description:':'Legal'},inplace=True)

In [ ]:
single_family_df['ListingPrice'].describe()

In [ ]:
single_family_df.loc[single_family_df.ListingPrice <10000]

In [ ]:
single_family_df = single_family_df[single_family_df.ListingPrice > 1000]

## 1.10 Bedrooms<a id='1.10_Bedrooms'></a>

In [ ]:
single_family_df[['Bedrooms:','Bedroom:','Primary Bedroom:']]

In [ ]:
single_family_df[['Bedrooms:','Bedroom:','Primary Bedroom:','Average Bedrooms:']].isna().sum()

First I calculate area for Bedroom and Primary Bedroom separately then fill null values for with the median of area per subdivision and 0 for bedroom and primary bedroom respectively. then add them as total bedrooms in Sqft. 

In [ ]:
single_family_df['Primary_Bedroom_clean'] = single_family_df['Primary Bedroom:'].str.split(',')
single_family_df['Primary_Bedroom_clean'] = single_family_df['Primary_Bedroom_clean'].apply(area_calc)
single_family_df.update(single_family_df[['Primary_Bedroom_clean']].fillna(0))

single_family_df['Bedroom_clean'] = single_family_df['Bedroom:'].str.split(',')
single_family_df['Bedroom_clean'] = single_family_df['Bedroom_clean'].apply(area_calc)
single_family_df['Bedroom_clean'] = single_family_df.groupby('SubName')['Bedroom_clean'].transform(lambda x: x.fillna(x.median()))

In [ ]:
single_family_df['TotalBedSqft'] = single_family_df['Bedroom_clean'] + single_family_df['Primary_Bedroom_clean']

In [ ]:
pd.options.display.max_colwidth = 100
single_family_df[['Bedrooms:','Bedroom:','Primary Bedroom:','Primary_Bedroom_clean','Bedroom_clean','TotalBedSqft']].sample(20,random_state=100)

In [ ]:
single_family_df[single_family_df.TotalBedSqft==0][['Year Built:','Bedroom:','Primary Bedroom:','TotalBedSqft']]

In [ ]:
single_family_df['TotalBedSqft'] = single_family_df.groupby('SubName')['TotalBedSqft'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
single_family_df[single_family_df.TotalBedSqft==0][['Year Built:','Bedroom:','Primary Bedroom:','TotalBedSqft']]

In [ ]:
single_family_df = single_family_df[single_family_df.TotalBedSqft!=0]

In [ ]:
single_family_df[single_family_df.TotalBedSqft>1500][['Bedrooms:','Bedroom:','Primary Bedroom:','TotalBedSqft','Address','Building Sqft.:']]

In [ ]:
# single_family_df.loc[153,'TotalBedSqft'] = 484.8
# single_family_df.loc[196,'TotalBedSqft'] = 506
# single_family_df.loc[781,'TotalBedSqft'] = 365.25
# single_family_df.loc[3687,'TotalBedSqft'] = 395.6
# single_family_df.loc[4384,'TotalBedSqft'] = 550
# single_family_df.loc[4390,'TotalBedSqft'] = 676
# single_family_df.loc[4446,'TotalBedSqft'] = 1747
# single_family_df.loc[5083,'TotalBedSqft'] = 696
# single_family_df.loc[6437,'TotalBedSqft'] = 432
# single_family_df.loc[6566,'TotalBedSqft'] = 1915
# single_family_df.loc[6891,'TotalBedSqft'] = 653
# single_family_df.loc[7505,'TotalBedSqft'] = 675.521
# single_family_df.loc[7733,'TotalBedSqft'] = 820
# single_family_df.loc[10176,'TotalBedSqft'] = 674
# single_family_df.loc[10218,'TotalBedSqft'] = 655
# single_family_df.loc[10849,'TotalBedSqft'] = 496

In [ ]:
single_family_df['Bedrooms:'].isnull().sum()

There are 20 houses with no information about bedrooms and I will drop those rows.

In [ ]:
single_family_df=single_family_df[~single_family_df['Bedrooms:'].isnull()]

In [ ]:
single_family_df['Bedrooms:']=single_family_df['Bedrooms:'].str.split(' ').str[0]
single_family_df['Bedrooms:']=single_family_df['Bedrooms:'].astype(int)
single_family_df.rename(columns = {'Bedrooms:':'NoBed'},inplace=True)
single_family_df['NoBed'].describe()

In [ ]:
single_family_df['Average Bedrooms:'].isna().sum()

In [ ]:
single_family_df.groupby('SubName')['Average Bedrooms:'].value_counts()

In [ ]:
single_family_df['Average Bedrooms:'].loc[single_family_df.SubName=='Wortham Park']

It seems average bedroom is same for each subdivision. We can fill null values with the mean. 

In [ ]:
single_family_df['Average Bedrooms:'] = single_family_df.groupby('SubName')['Average Bedrooms:'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
single_family_df['Average Bedrooms:'].isna().sum()

So there is no information for those subdivisions. lets drop these rows

In [ ]:
single_family_df=single_family_df[~single_family_df['Average Bedrooms:'].isnull()]
single_family_df.rename(columns ={'Average Bedrooms:':'AvgBed'},inplace=True)

In [ ]:
single_family_df.drop(['Bedroom:','Primary Bedroom:','Primary_Bedroom_clean','Bedroom_clean'],axis=1,inplace=True)

## 1.11 Bathrooms<a id='1.11_Bathrooms'></a>

In [ ]:
single_family_df['Baths:'].isnull().sum()

In [ ]:
single_family_df=single_family_df[~single_family_df['Baths:'].isnull()]

In [ ]:
single_family_df[['Baths:']]

In [ ]:
single_family_df['FullBath']=single_family_df['Baths:'].str.split(' ').str[0].astype(int)

In [ ]:
single_family_df['FullBath']

In [ ]:
No_Bath = single_family_df['Baths:'].str.split('&').str[1].str.strip()
No_Bath.fillna('0',inplace=True) 
single_family_df['HalfBath']=[int(item[0]) for item in No_Bath.str.split(' ')]
single_family_df['HalfBath'].replace(',','',inplace=True)
single_family_df[['FullBath','HalfBath']].info()

In [ ]:
single_family_df.drop('Baths:',axis=1,inplace=True)

## 1.12 Stories<a id='1.12_Stories'></a>

In [365]:
single_family_df['Stories:'].value_counts()

2.0       3820
1.0       3141
3.0        400
1.5        190
4.0         54
2.5         11
2576.0       1
5.0          1
Name: Stories:, dtype: int64

There is one house with 2576 stories which I believe it is a typo. Lets take look at this home.

In [366]:
single_family_df['Stories:'].isnull().sum()

1

In [367]:
single_family_df=single_family_df[~single_family_df['Stories:'].isnull()]

In [368]:
single_family_df.rename(columns ={'Stories:':'Stories'},inplace=True)
single_family_df['Stories']=pd.to_numeric(single_family_df['Stories'])

In [369]:
single_family_df['Address'].loc[single_family_df['Stories']>4]

539        5334 Calle Cadiz Place
10097    22614 Auburn Valley Lane
Name: Address, dtype: object

The second one is an apartment and second one is 2 stories house. know we replace 2576 with 2.

In [370]:
single_family_df['Stories'].loc[single_family_df['Stories']==2576]=2

In [371]:
single_family_df['Stories'].value_counts()

2.0    3821
1.0    3141
3.0     400
1.5     190
4.0      54
2.5      11
5.0       1
Name: Stories, dtype: int64

Since we are looking for single family homes, those apartment are different from what we are looking for. so, images for those houses would be different. I will keep the houses with have equal or less than 3 stories.

In [372]:
single_family_df = single_family_df[single_family_df.Stories < 4]
single_family_df['Stories'].value_counts()

2.0    3821
1.0    3141
3.0     400
1.5     190
2.5      11
Name: Stories, dtype: int64

## 1.13 Style<a id='1.13_Style'></a>

In [373]:
single_family_df['Style:'].value_counts()

Traditional                                           5328
Contemporary/Modern                                    645
Ranch                                                  320
Contemporary/Modern,Traditional                        227
Other Style                                            163
                                                      ... 
Georgian,Victorian                                       1
Contemporary/Modern,Mediterranean,Spanish,Traditio       1
Contemporary/Modern,English                              1
Contemporary/Modern,French,Mediterranean,Tradition       1
Colonial,Contemporary/Modern,Victorian                   1
Name: Style:, Length: 76, dtype: int64

In [374]:
single_family_df['Style:'].isnull().sum()

0

In [375]:
single_family_df.rename(columns ={'Style:':'Style'},inplace=True)

In [376]:
single_family_df['Style'].unique()

array(['Contemporary/Modern', 'Traditional', 'Other Style',
       'Contemporary/Modern,Traditional', 'Split Level', 'Colonial',
       'Mediterranean,Spanish', 'French', 'Mediterranean,Traditional',
       'Ranch', 'Mediterranean', 'Other Style,Traditional',
       'Contemporary/Modern,French', 'Contemporary/Modern,English,French',
       'Contemporary/Modern,Ranch,Traditional', 'Georgian,Traditional',
       'French,Traditional', 'Spanish', 'Spanish,Traditional',
       'Colonial,Georgian', 'English,Traditional', 'Colonial,Traditional',
       'Georgian', 'Other Style,Ranch',
       'Contemporary/Modern,French,Traditional', 'Victorian',
       'Traditional,Victorian', 'Contemporary/Modern,English,Traditional',
       'Colonial,Contemporary/Modern',
       'Contemporary/Modern,Mediterranean,Traditional',
       'Contemporary/Modern,Mediterranean', 'English',
       'Ranch,Traditional', 'Colonial,Ranch,Traditional',
       'Other Style,Traditional,Victorian',
       'Contemporary/Moder

In [377]:
standard_style=['Contemporary/Modern','Traditional','Mediterranean','Colonial','Ranch','Split Level','Other']
#For each correct roof  type . in standard roof list
for style in standard_style:
    
    # Find matches in gender
    matches = process.extract(style, single_family_df.Style,
                 limit = single_family_df.shape[0])
    
    
# For each possible_match with similarity score >= 90
    for possible_match in matches:
        if possible_match[1] >= 90:
      
            
            matching = single_family_df.Style == possible_match[0]
           # I decided to use 'W' for female since there is high similarity between 'female' and 'male' 
        single_family_df.loc[matching , 'Style'] = style

## 1.14 Year Built<a id='1.14_Year_Built'></a>

In [378]:
single_family_df['Year Built:'].isnull().sum()

31

In [379]:
single_family_df['Year Built:'].value_counts()

2020   / Builder               955
2006   / Appraisal District    180
2005   / Appraisal District    144
2014   / Appraisal District    135
1955   / Appraisal District    131
                              ... 
2004   / Seller                  1
1942   / Appraisal               1
1985   / Appraisal               1
1912   / Builder                 1
1991   / Seller                  1
Name: Year Built:, Length: 278, dtype: int64

In [380]:
single_family_df=single_family_df[~single_family_df['Year Built:'].isnull()]
single_family_df['Year Built:']=single_family_df['Year Built:'].apply(lambda x:str(x).split(' ')[0])
single_family_df['Year Built:']=pd.to_datetime(single_family_df['Year Built:'],format='%Y').dt.year
single_family_df['Year Built:'].value_counts()

2020    1004
2006     204
2015     172
2014     166
2005     161
        ... 
1922       1
1934       1
1875       1
1907       1
1896       1
Name: Year Built:, Length: 113, dtype: int64

Those houses with year build of 2020 mostly under construction and there is no images from inside or even outside. I will drop those houses too.

In [381]:
single_family_df.rename(columns ={'Year Built:':'YearBuilt'},inplace=True)

In [382]:
single_family_df = single_family_df[single_family_df.YearBuilt < 2020]
single_family_df['YearBuilt'].value_counts()

2006    204
2015    172
2014    166
2005    161
1960    144
       ... 
1916      1
1908      1
1904      1
1900      1
1896      1
Name: YearBuilt, Length: 111, dtype: int64

## 1.15 Building Sqft<a id='1.15_Building_Sqft'></a>

In [383]:
single_family_df['Building Sqft.:']

13                  1,736161(mÂ²)  /Builder
20       2,671248(mÂ²)  /Appraisal District
28       1,972183(mÂ²)  /Appraisal District
29       1,688157(mÂ²)  /Appraisal District
30       1,410131(mÂ²)  /Appraisal District
                        ...                
11029       83978(mÂ²)  /Appraisal District
11030       92386(mÂ²)  /Appraisal District
11031    1,563145(mÂ²)  /Appraisal District
11032    1,461136(mÂ²)  /Appraisal District
11033    1,348125(mÂ²)  /Appraisal District
Name: Building Sqft.:, Length: 6458, dtype: object

In [384]:
single_family_df['Building Sqft.:'].isnull().sum()

11

In [385]:
single_family_df=single_family_df[~single_family_df['Building Sqft.:'].isnull()]
single_family_df['Building Sqft.:']=single_family_df['Building Sqft.:'].apply(lambda x:x.split(',')[0] + x.split(',')[1][0:3] if ',' in x else x[0:3])
single_family_df['Building Sqft.:']=single_family_df['Building Sqft.:'].str.replace(',','')
single_family_df['Building Sqft.:']=pd.to_numeric(single_family_df['Building Sqft.:'])
single_family_df.rename(columns ={'Building Sqft.:':'BuildSqft'},inplace=True)

In [386]:
single_family_df.BuildSqft.describe()

count     6447.000000
mean      2824.024973
std       1332.015240
min        400.000000
25%       1936.000000
50%       2550.000000
75%       3406.000000
max      21032.000000
Name: BuildSqft, dtype: float64

## 1.16 Lot Size<a id='1.16_Lot_Size'></a>

In [387]:
single_family_df['Lot Size:'].isnull().sum()

48

In [388]:
single_family_df=single_family_df[~single_family_df['Lot Size:'].isnull()]

In [389]:
single_family_df['Lot Size:']

13       1,428 Sqft.133(mÂ²)  /Appraisal District
20       2,006 Sqft.186(mÂ²)  /Appraisal District
28       5,000 Sqft.465(mÂ²)  /Appraisal District
29       1,918 Sqft.178(mÂ²)  /Appraisal District
30       1,845 Sqft.171(mÂ²)  /Appraisal District
                           ...                   
11029    7,100 Sqft.660(mÂ²)  /Appraisal District
11030    7,100 Sqft.660(mÂ²)  /Appraisal District
11031    7,100 Sqft.660(mÂ²)  /Appraisal District
11032    7,200 Sqft.669(mÂ²)  /Appraisal District
11033    7,100 Sqft.660(mÂ²)  /Appraisal District
Name: Lot Size:, Length: 6399, dtype: object

In [390]:
single_family_df['Lot Size:']=single_family_df['Lot Size:'].str.replace(',','')
single_family_df['Lot Size:']=single_family_df['Lot Size:'].apply(lambda x:float(x.split(' ')[0])*43560 if 'Acres' in 
                                                                  x else float(x.split(' ')[0]))

# single_family_df['Lot Size:']=pd.to_numeric(single_family_df['Lot Size:'])
single_family_df.rename(columns ={'Lot Size:':'LotSize'},inplace=True)

In [391]:
single_family_df.LotSize.describe()

count    6.399000e+03
mean     1.133162e+04
std      2.817675e+04
min      0.000000e+00
25%      6.446000e+03
50%      8.160000e+03
75%      1.020050e+04
max      1.306800e+06
Name: LotSize, dtype: float64

the minimum lot size is 0 which need to investigate.

In [392]:
single_family_df[single_family_df.LotSize <1000]

,image_link,ListingPrice,Address,City:,ZipCode,County,Legal,NoBed,Stories,Style,...,MedianApp,MedianYearBlt,MedianSqft,AvgNeighborValRange,MedianPrice/Sqft,PaidTax,TaxRate,TotalBedSqft,FullBath,HalfBath
3267,"['https://photos.harstatic.com/191172097/hr/img-1.jpeg?ts=2020-11-10T18:10:20.193', 'https://pho...",339000,9609 Fannin Station E,Houston,77045,Harris County,LT 52 BLK 1 FANNIN STATION SEC 1,3,3.0,Contemporary/Modern,...,349985,2013,1383,371500.0,166.43,8636.0,2.4216,485.479167,3,1
4765,"['https://photos.harstatic.com/190806088/hr/img-1.jpeg?ts=2020-10-25T08:51:53.280', 'https://pho...",173550,7807 Vickijohn,Houston,77071,Harris County,TR 2 BLK 2 FONDREN SW NORTHFIELD SEC 1,3,1.0,Contemporary/Modern,...,214780,1977,8740,229000.0,84.39,2743.0,2.6237,528.000000,2,0


There is one house with lot size less than 1000. lest drop it.

In [393]:
single_family_df = single_family_df[single_family_df.LotSize >= 1000]
single_family_df.LotSize.describe()

count    6.397000e+03
mean     1.133501e+04
std      2.818050e+04
min      1.104000e+03
25%      6.448000e+03
50%      8.160000e+03
75%      1.020100e+04
max      1.306800e+06
Name: LotSize, dtype: float64

## 1.17 Maintenance Fee<a id='1.17_Maintenance_Fee'></a>

In [394]:
single_family_df['Maintenance Fee:']

13       ['$ 1195 / Annually', 'Mandatory / $1195 / Annually']
20       ['$ 2244 / Annually', 'Mandatory / $2244 / Annually']
28                                                          No
29       ['$ 1500 / Annually', 'Mandatory / $1500 / Annually']
30       ['$ 1400 / Annually', 'Mandatory / $1400 / Annually']
                                 ...                          
11029                                                       No
11030                                                       No
11031                                                       No
11032                                                       No
11033                                                       No
Name: Maintenance Fee:, Length: 6397, dtype: object

In [395]:
single_family_df['Maintenance Fee:'].isnull().sum()

16

In [396]:
single_family_df['Maintenance Fee:'].value_counts()

No                                                       1510
['$ 650 / Annually', 'Mandatory / $650 / Annually']       125
['$ 450 / Annually', 'Mandatory / $450 / Annually']       124
['$ 350 / Annually', 'Mandatory / $350 / Annually']       118
['$ 400 / Annually', 'Mandatory / $400 / Annually']       100
                                                         ... 
['$ 105 / Annually', 'Mandatory / $105 / Annually']         1
['$ 405 / Annually', 'Mandatory / $405 / Annually']         1
['$ 969 / Annually', 'Mandatory / $969 / Annually']         1
['$ 698 / Annually', 'Mandatory / $698 / Annually']         1
['$ 2760 / Annually', 'Mandatory / $2760 / Annually']       1
Name: Maintenance Fee:, Length: 814, dtype: int64

In [397]:
single_family_df['Maintenance Fee:'].isin(['No','No / $0','Voluntary / Annually','Voluntary /0/ Annually']).sum()

1557

In [398]:
def MaintenanceFee(fee):
    """This function calculates the Maintenance fee for this column"""
    if type(fee) != float:
        fee = fee.split(',')[0].split('/')
        if fee[0].strip() in ['No','No / $0','Voluntary / Annually','Voluntary /0/ Annually','Voluntary','Mandatory']:
            fee = 0
        elif len(fee) > 1 and fee[1].replace('\'','').strip() in ['Annually']:
            fee = float(fee[0].replace('$','').replace('\'','').replace(' ','').replace('[',''))
        elif len(fee) > 1 and fee[1].replace('\'','').strip() in ['Quarterly']:   
            fee = float(fee[0].replace('$','').replace('\'','').replace(' ','').replace('[',''))*4
        elif len(fee) > 1 and fee[1].replace('\'','').strip() in ['Monthly']:   
            fee = float(fee[0].replace('$','').replace('\'','').replace(' ','').replace('[',''))*12 
        else:
            fee = float(fee[0].replace('$','').replace('\'','').replace(' ','').replace('[',''))
    else:
        fee = 0
    return fee

In [399]:
single_family_df['MaintenanceFee'] = single_family_df['Maintenance Fee:'].apply(MaintenanceFee)

In [400]:
single_family_df.drop('Maintenance Fee:',axis=1,inplace=True)

## 1.18 Fireplace<a id='1.18_Fireplace'></a>

In [401]:
single_family_df['Fireplace:'].value_counts()

1/Gaslog Fireplace                                                          1154
1                                                                            741
1/Gas Connections                                                            653
1/Wood Burning Fireplace                                                     463
1/Gas Connections, Gaslog Fireplace                                          340
                                                                            ... 
2/Stove                                                                        1
1/Freestanding, Stove                                                          1
/Stove                                                                         1
1/Freestanding, Mock Fireplace                                                 1
3/Freestanding, Gaslog Fireplace, Mock Fireplace, Wood Burning Fireplace       1
Name: Fireplace:, Length: 83, dtype: int64

In [402]:
pd.Series([str(x)[0]  for x in single_family_df['Fireplace:'] if x is not None]).value_counts()

1    4046
n    1736
2     443
3      99
/      39
4      23
5      10
6       1
dtype: int64

In [403]:
single_family_df['Fireplace:']=single_family_df['Fireplace:'].apply(lambda x:int(str(x)[0]) if str(x)[0]
                                                                    in ['1','2','3','4','5','6','7'] else 0)

In [404]:
single_family_df['Fireplace:'].value_counts()

1    4046
0    1775
2     443
3      99
4      23
5      10
6       1
Name: Fireplace:, dtype: int64

## 1.19 HOA Mandatory<a id='1.19_HOA_Mandatory'></a>

In [405]:
single_family_df['HOA Mandatory:']

13       Yes
20       Yes
28        No
29       Yes
30       Yes
        ... 
11029     No
11030     No
11031     No
11032     No
11033     No
Name: HOA Mandatory:, Length: 6397, dtype: object

We can fill null values for HOA by 'NO'.

In [406]:
single_family_df['HOA Mandatory:'][single_family_df['HOA Mandatory:'].isnull()]='No'
single_family_df.rename(columns ={'HOA Mandatory:':'HOA'},inplace=True)

In [407]:
missing = missing_cal(single_family_df)
missing.loc[missing['%'] > 0].sort_values(by="%",ascending=False)

,count,%
Median Appraised Value / Square ft.:,6265,97.936533
Legal,1,0.015632


Median Appraised Value / Square ft.: has 100% missing value we can drop it and also drop rows with null value for legal.

In [408]:
single_family_df.drop('Median Appraised Value / Square ft.:',axis = 1, inplace=True)
single_family_df=single_family_df[~single_family_df['Legal'].isnull()]
single_family_df=single_family_df[~single_family_df['TotalBedSqft'].isnull()]

lets take a look at dataset to see if we need to rename some columns:

In [409]:
single_family_df.columns

Index(['image_link', 'ListingPrice', 'Address', 'City:', 'ZipCode', 'County',
       'Legal', 'NoBed', 'Stories', 'Style', 'YearBuilt', 'BuildSqft',
       'LotSize', 'Fireplace:', 'Heating:', 'Cooling:', 'Ice Maker:',
       'Microwave:', 'Compactor:', 'Dishwasher:', 'Disposal:', 'Roof:',
       'Foundation:', 'Private Pool:', 'Exterior Type:', 'Lot Description:',
       'Controlled Access:', 'Water Sewer:', 'Unit Location:', 'Area Pool:',
       'Dwelling Type:', 'HOA', 'List Type:', 'Other Fees:', 'AvgBed',
       'Average Baths:', 'Carport Description:', 'UtilitySqft', 'StudySqft',
       'GameSqft', 'BreakfastSqft', 'Garage', 'LivingSqft', 'DiningSqft',
       'KitchenSqft', 'SubName', 'MedianApp', 'MedianYearBlt', 'MedianSqft',
       'AvgNeighborValRange', 'MedianPrice/Sqft', 'PaidTax', 'TaxRate',
       'TotalBedSqft', 'FullBath', 'HalfBath', 'MaintenanceFee'],
      dtype='object')

In [410]:
single_family_df.reset_index(inplace=True,drop=True)

In [411]:
single_family_df.rename(columns = {'City:':'City','Fireplace:':'Fireplace', 'Heating:':'Heating', 'Cooling:':'Cooling',
                                   'Ice Maker:':'IceMaker', 'Microwave:':'Microwave','Compactor:':'Compactor',
                                   'Dishwasher:':'Dishwasher','Disposal:':'Disposal','Roof:':'Roof',
                                   'Foundation:':'Foundation','Private Pool:':'PrivatePool',
                                   'Exterior Type:':'ExteriorType','Lot Description:':'LotDes',
                                   'Controlled Access:':'ControlAccess','Water Sewer:':'WaterSewer',
                                   'Unit Location:':'UnitLoc','Area Pool:':'AreaPool','Dwelling Type:':'DwellingType',
                                   'List Type:':'ListType','Other Fees:':'OtherFees','Average Baths:':'AvgBaths',
                                   'Carport Description:':'CarportDescription'},inplace=True)

In [412]:
single_family_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6396 entries, 0 to 6395
Data columns (total 57 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   image_link           6396 non-null   object 
 1   ListingPrice         6396 non-null   int64  
 2   Address              6396 non-null   object 
 3   City                 6396 non-null   object 
 4   ZipCode              6396 non-null   int64  
 5   County               6396 non-null   object 
 6   Legal                6396 non-null   object 
 7   NoBed                6396 non-null   int32  
 8   Stories              6396 non-null   float64
 9   Style                6396 non-null   object 
 10  YearBuilt            6396 non-null   int64  
 11  BuildSqft            6396 non-null   int64  
 12  LotSize              6396 non-null   float64
 13  Fireplace            6396 non-null   int64  
 14  Heating              6396 non-null   object 
 15  Cooling              6396 non-null   o

## 1.20 Other Fees<a id='1.20_Other_Fees'></a>

In [413]:
single_family_df.OtherFees.value_counts()

No                                                                    2873
Yes / 250 / Transfer Fee                                               401
Yes / 250 / Transfer                                                   155
Yes / 250                                                              143
Yes / 200 / Transfer Fee                                               121
                                                                      ... 
Yes / 105                                                                1
Yes / 325 / transfer                                                     1
Yes / 95+500 / transfer fee + trails                                     1
Yes / 300 / Walls, gates, monuments, lights, common area landscapi       1
Yes / $150 / HOA Transfer Fee                                            1
Name: OtherFees, Length: 1101, dtype: int64

We need to clean this column and to do so I will change No value with 0 and calculate total fees for the rest.

In [414]:
single_family_df.drop('OtherFees',axis = 1, inplace=True)

In [415]:
single_family_df.head()

,image_link,ListingPrice,Address,City,ZipCode,County,Legal,NoBed,Stories,Style,...,MedianYearBlt,MedianSqft,AvgNeighborValRange,MedianPrice/Sqft,PaidTax,TaxRate,TotalBedSqft,FullBath,HalfBath,MaintenanceFee
0,"['https://photos.harstatic.com/189115929/hr/img-1.jpeg?ts=2020-08-31T18:04:52.670', 'https://pho...",364990,1735 Eado Point Lane,Houston,77003,Harris County,LT 17 BLK 2 EADO POINT,3,3.0,Contemporary/Modern,...,2018,1485,217000.0,192.51,2169.0,2.6554,430.0,3,1,1195.0
1,"['https://photos.harstatic.com/188179029/hr/img-1.jpeg?ts=2020-08-03T08:56:34.177', 'https://pho...",419000,619 Live Oak Street,Houston,77003,Harris County,LT 7 BLK 1 CAPITOL OAKS SEC 3 RP NO 1,3,3.0,Traditional,...,2012,1563,443500.0,200.05,10366.0,2.5466,320.0,3,1,2244.0
2,"['https://photos.harstatic.com/189527871/hr/img-1.jpeg?ts=2020-09-14T06:08:01.320', 'https://pho...",298800,1417 Hussion Street,Houston,77003,Harris County,LT 7 BLK 1 W L EDMUNDSON SEC 3,3,1.0,Traditional,...,1929,5000,126500.0,147.06,5854.0,2.5716,616.0,1,1,0.0
3,"['https://photos.harstatic.com/189915458/hr/img-1.jpeg?ts=2020-09-26T08:22:44.337', 'https://pho...",289900,1737 Aden Drive,Houston,77003,Harris County,"Lot 59, Block 1",2,2.0,Other,...,2005,1904,259000.0,181.62,6685.4,2.5465,330.0,2,0,1500.0
4,"['https://photos.harstatic.com/190529396/hr/img-1.jpeg?ts=2020-10-26T13:11:52.120', 'https://pho...",284900,1773 Aden Drive,Houston,77003,Harris County,LT 90 BLK 1 MIDTOWN VILLAGE,2,2.0,Traditional,...,2005,1904,259000.0,181.62,6299.0,2.5466,282.0,2,1,1400.0


## 1.21 Roof<a id='1.21_Roof'></a>

In [416]:
single_family_df.Roof.unique()

array(['Composition', 'Aluminum', 'Aluminum, Other', 'Other',
       'Composition, Other', 'Tile', 'Built Up', 'Aluminum, Composition',
       'Composition, Slate', 'Slate', 'Wood Shingle', 'Composition, Tile',
       'Built Up, Composition', 'Aluminum, Composition, Other',
       'Other, Tile', "['Composition', 'Composition']",
       'Other, Wood Shingle', 'Aluminum, Slate',
       'Aluminum, Other, Wood Shingle', 'Composition, Wood Shingle'],
      dtype=object)

To reduce number of category in roof type column I create a list of standard roof type and will match each category to standard list, then replace it with higher score.

In [417]:
standard_roof=['Composition','Aluminum','Tile','Slate','Wood Shingle','Built Up','Other']
#For each correct roof  type . in standard roof list
for roof in standard_roof:
    
    # Find matches in gender
    matches = process.extract(roof, single_family_df.Roof,
                 limit = single_family_df.shape[0])
    
    
# For each possible_match with similarity score >= 90
    for possible_match in matches:
        if possible_match[1] >= 90:
      
            
            matching = single_family_df.Roof == possible_match[0]
           # I decided to use 'W' for female since there is high similarity between 'female' and 'male' 
        single_family_df.loc[matching , 'Roof'] = roof

In [418]:
single_family_df.Roof.unique()

array(['Composition', 'Aluminum', 'Other', 'Tile', 'Built Up', 'Slate',
       'Wood Shingle'], dtype=object)

## 1.21 Foundation<a id='1.21_Foundation'></a>

In [419]:
single_family_df.Foundation.unique()

array(['Slab', 'Pier & Beam', 'Block & Beam',
       'Slab, Slab on Builders Pier', 'Slab on Builders Pier',
       'Pier & Beam, Slab', 'Other, Slab',
       'Pier & Beam, Slab on Builders Pier', 'Other, Pier & Beam',
       'Other', 'Block & Beam, Pier & Beam', 'Block & Beam, Slab',
       'Block & Beam, Slab on Builders Pier', 'On Stilts',
       'Other, Slab on Builders Pier'], dtype=object)

In [420]:
standard_foundation=['Slab','Block & Beam','Pier & Beam','On Stilts','Other']
for found in standard_foundation:
    
    # Find matches in gender
    matches = process.extract(found, single_family_df.Foundation,
                 limit = single_family_df.shape[0])
    
    
# For each possible_match with similarity score >= 90
    for possible_match in matches:
        if possible_match[1] >= 90:
      
            
            matching = single_family_df.Foundation == possible_match[0]
           # I decided to use 'W' for female since there is high similarity between 'female' and 'male' 
        single_family_df.loc[matching , 'Foundation'] = found
print(single_family_df.Foundation.unique())

['Slab' 'Pier & Beam' 'Block & Beam' 'Other' 'On Stilts']


In [421]:
single_family_df.ExteriorType.unique()

array(['Cement Board', 'Cement Board, Stucco', 'Brick, Cement Board',
       'Brick, Vinyl, Wood', 'Other, Stucco', 'Stucco', 'Brick, Wood',
       'Wood', 'Brick, Cement Board, Wood', 'Aluminum, Brick, Vinyl',
       'Brick, Other', 'Brick & Wood', 'Brick, Stucco', 'Brick',
       'Brick, Stone, Wood', 'Brick & Wood, Cement Board, Wood',
       'Cement Board, Stone, Stucco', 'Other', 'Stone, Stucco',
       'Aluminum, Stone, Stucco, Wood', 'Stone & Wood, Stucco',
       'Brick Veneer, Stone', 'Brick, Other, Wood',
       'Cement Board, Stone, Wood', 'Brick Veneer',
       'Brick & Wood, Brick, Cement Board', 'Brick, Stone, Stucco',
       'Brick & Wood, Brick', 'Asbestos', 'Unknown', 'Brick, Vinyl',
       'Asbestos, Wood', 'Brick, Stone, Stucco, Vinyl, Wood',
       'Aluminum, Vinyl', 'Stone', 'Cement Board, Wood', 'Aluminum',
       'Stucco, Wood', 'Brick & Wood, Cement Board', 'Aluminum, Stone',
       'Aluminum, Brick, Vinyl, Wood', 'Aluminum, Other',
       'Brick & Wood, Stone &

In [422]:
single_family_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6396 entries, 0 to 6395
Data columns (total 56 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   image_link           6396 non-null   object 
 1   ListingPrice         6396 non-null   int64  
 2   Address              6396 non-null   object 
 3   City                 6396 non-null   object 
 4   ZipCode              6396 non-null   int64  
 5   County               6396 non-null   object 
 6   Legal                6396 non-null   object 
 7   NoBed                6396 non-null   int32  
 8   Stories              6396 non-null   float64
 9   Style                6396 non-null   object 
 10  YearBuilt            6396 non-null   int64  
 11  BuildSqft            6396 non-null   int64  
 12  LotSize              6396 non-null   float64
 13  Fireplace            6396 non-null   int64  
 14  Heating              6396 non-null   object 
 15  Cooling              6396 non-null   o

## 1.22 Dwelling Type<a id='1.22_Dwelling_Type'></a>

In [423]:
single_family_df.DwellingType.unique()

array(['Free Standing', 'Free Standing, Patio Home', 'Duplex',
       'Free Standing, Historic', 'Historic', 'Patio Home',
       'Duplex, Historic', 'Duplex, Free Standing',
       'Duplex, Free Standing, Patio Home', 'Manufactured',
       'Duplex, Free Standing, Manufactured, Patio Home'], dtype=object)

In [424]:
standard_dwelling=['Free Standing','Patio Home','Duplex','Historic']
for types in standard_dwelling:
    
    
    matches = process.extract(types, single_family_df.DwellingType,
                 limit = single_family_df.shape[0])
    
    
# For each possible_match with similarity score >= 90
    for possible_match in matches:
        if possible_match[1] >= 90:
      
            
            matching = single_family_df.DwellingType == possible_match[0]
            
        single_family_df.loc[matching , 'DwellingType'] = types
print(single_family_df.DwellingType.unique())

['Free Standing' 'Duplex' 'Historic' 'Patio Home' 'Manufactured']


In [425]:
single_family_df.to_csv('../data/processed/SingleFamily.csv',index=False)